In [17]:
import pandas as pd
import matplotlib.pyplot as plt

Cleaning FAO Food Loss from 2000 to 2021 dataset

In [18]:
# Carregando e limpando o primeiro dataset
df1 = pd.read_csv("../datasets/FAO_food_loss_2000_2021.csv")
columns_to_remove = ["m49_code", "region", "cpc_code", "loss_percentage_original",
                     "sample_size", "method_data_collection", "reference",
                      "reference.1", "reference.2", "loss_quantity", "treatment", "cause_of_loss"]
df1 = df1.drop(columns=columns_to_remove)


Cleaning Crops All Data Normalized Dataset

In [19]:
# Carregando e limpando o segundo dataset
df2 = pd.read_csv("../datasets/Crops_AllData_Normalized.csv", encoding='latin1')
columns_to_remove = ["Area Code", "Item Code", "Element Code", "Year Code", "Flag"]
df2 = df2.drop(columns=columns_to_remove)
df2 = df2[df2["Element"] == "Production"]
df2 = df2.rename(columns={"Area": "country", "Item": "commodity", "Year": "year"})

df2

,country,commodity,Element,year,Unit,Value
89,Afghanistan,"Almonds, with shell",Production,1975,tonnes,0.0
90,Afghanistan,"Almonds, with shell",Production,1976,tonnes,9800.0
91,Afghanistan,"Almonds, with shell",Production,1977,tonnes,9000.0
92,Afghanistan,"Almonds, with shell",Production,1978,tonnes,12000.0
93,Afghanistan,"Almonds, with shell",Production,1979,tonnes,10500.0
...,...,...,...,...,...,...
2513863,Net Food Importing Developing Countries,Vegetables Primary,Production,2015,tonnes,81811964.0
2513864,Net Food Importing Developing Countries,Vegetables Primary,Production,2016,tonnes,82352673.0
2513865,Net Food Importing Developing Countries,Vegetables Primary,Production,2017,tonnes,83630787.0
2513866,Net Food Importing Developing Countries,Vegetables Primary,Production,2018,tonnes,86033417.0


Cleaning the GDP dataset

In [20]:
# Carregando e limpando o terceiro dataset
df3 = pd.read_csv("../datasets/data_20240314121256.csv")
columns_to_remove = ["IndicatorCode", "IndicatorName", "VariableName", "MeasurementName", "CountryCode", "Alpha3Code"]
df3 = df3.drop(columns=columns_to_remove)
df3 = df3.rename(columns={"CountryName": "country", "PeriodCode": "year", "Value": "GDP"})

df3 = df3.drop(columns=["Unnamed: 9"])

df3

,country,year,GDP
0,Albania,2005,5865.3
1,Albania,2006,6565.6
2,Albania,2007,7284.1
3,Albania,2008,8228.3
4,Albania,2009,8813.5
...,...,...,...
1003,Uzbekistan,2018,7422.5
1004,Uzbekistan,2019,7899.6
1005,Uzbekistan,2020,8057.1
1006,Uzbekistan,2021,8931.4


Merging the 3 datasets

In [21]:
# # Junção dos datasets df1 e df2 utilizando o método 'outer'
merged_df = pd.merge(df1, df2, how="outer", on=["country", "commodity", "year"])
merged_df = merged_df.drop(columns=["Unit", "Element"])

# # Junção do dataset resultante com o df3, também utilizando o método 'outer'
merged_df = pd.merge(merged_df, df3, how="outer", on=["country", "year"])
merged_df = merged_df.rename(columns={"commodity": "product", "Value": "country_product_prodution"})

merged_df

,country,product,year,loss_percentage,activity,food_supply_stage,country_product_prodution,GDP
0,Myanmar,"Groundnuts, excluding shelled",2009,5.22,NaN,Whole supply chain,NaN,NaN
1,Myanmar,Areca nuts,2009,NaN,NaN,NaN,115800.0,NaN
2,Myanmar,Barley,2009,NaN,NaN,NaN,NaN,NaN
3,Myanmar,"Bastfibres, other",2009,NaN,NaN,NaN,11600.0,NaN
4,Myanmar,"Beans, dry",2009,NaN,NaN,NaN,3375000.0,NaN
...,...,...,...,...,...,...,...,...
904940,United States,NaN,2021,NaN,NaN,NaN,NaN,70686.7
904941,United States,NaN,2022,NaN,NaN,NaN,NaN,76585.2
904942,Uzbekistan,NaN,2020,NaN,NaN,NaN,NaN,8057.1
904943,Uzbekistan,NaN,2021,NaN,NaN,NaN,NaN,8931.4


Data Type formalization

In [22]:
merged_df['GDP'] = pd.to_numeric(merged_df['GDP'], errors='coerce')
merged_df['country'] = merged_df['country'].astype(str)
merged_df['product'] = merged_df['product'].astype(str)
merged_df['activity'] = merged_df['activity'].astype(str)
merged_df['food_supply_stage'] = merged_df['food_supply_stage'].astype(str)
merged_df["GDP"] = pd.to_numeric(merged_df['GDP'], errors='coerce')

Saving and visualizing the last dataframe

In [23]:
merged_df.to_csv("../datasets/dataset.csv", index=False)

#Carregar e visualizar o dataset final
df = pd.read_csv("../datasets/dataset.csv")

#print(df.dtypes)
#df.isna().sum()

df = df[df['country'] != 'world']

df

/Users/joaopino/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3526: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,country,product,year,loss_percentage,activity,food_supply_stage,country_product_prodution,GDP
0,Myanmar,"Groundnuts, excluding shelled",2009,5.22,NaN,Whole supply chain,NaN,NaN
1,Myanmar,Areca nuts,2009,NaN,NaN,NaN,115800.0,NaN
2,Myanmar,Barley,2009,NaN,NaN,NaN,NaN,NaN
3,Myanmar,"Bastfibres, other",2009,NaN,NaN,NaN,11600.0,NaN
4,Myanmar,"Beans, dry",2009,NaN,NaN,NaN,3375000.0,NaN
...,...,...,...,...,...,...,...,...
904940,United States,NaN,2021,NaN,NaN,NaN,NaN,70686.7
904941,United States,NaN,2022,NaN,NaN,NaN,NaN,76585.2
904942,Uzbekistan,NaN,2020,NaN,NaN,NaN,NaN,8057.1
904943,Uzbekistan,NaN,2021,NaN,NaN,NaN,NaN,8931.4
